# Test RNN sur le déjeuner diner 
sans prise en compte de l'identité utilisateur pour les repas 

Source utilisée pour la programmation dans ce notebook :
"Module 4 : RNN et attention pour le texte", Auteur: Vincent Guigue
https://github.com/vguigue/tuto_deep/blob/main/notebooks/4_1-rnn-gen.ipynb

In [1]:
# !pip install torchtext
from packaging import version
from pathlib import Path
from itertools import chain
from typing import Iterable, List, Tuple

import shutil
import torchtext
assert version.Version(torchtext.__version__) >= version.Version("0.9.0")
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import vocab, FastText
from collections import Counter
from tqdm.autonotebook import tqdm
from torch.nn.utils.rnn import pad_sequence
import sentencepiece as spm
from IPython.display import display, HTML
import os


cachepath = os.path.expanduser('~/.local/data')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# outils avancés de gestion des chemins
BASEPATH = Path("xp/generation")
TB_PATH =  BASEPATH / "logs"
TB_PATH.mkdir(parents=True, exist_ok=True)

# usage externe de tensorboard: (1) lancer la commande dans une console; (2) copier-coller l'URL dans un navigateur
display(HTML("<h2>Informations</h2><div>Pour visualiser les logs, tapez la commande : </div>"))
print(f"tensorboard --logdir {Path(TB_PATH).absolute()}")

C:\Users\jacqueno\AppData\Local\Temp\ipykernel_10428\2790905849.py:20: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


tensorboard --logdir C:\Users\jacqueno\Documents\IODAA\EXERSYS\Jup\selection_fin\RNN\xp\generation\logs


In [2]:
# !pip install sentencepiece
# !pip install datasets
# # !pip install fsspec==2021.8.1
# !pip uninstall huggingface-hub

# A. Chargement des données

In [3]:
#ADAPTATION EXERSYS 
import pickle
with open("C:/Users/jacqueno/Documents/IODAA/EXERSYS/Jup/REP_vraisalimfreqliste.pkl", "rb") as fichier:
    REP_vraisalimfreqliste = pickle.load(fichier)

In [4]:
#ADAPTATION EXERSYS 
print(REP_vraisalimfreqliste[:20])

[['28301', '9811', '7001', '18008', '19611', '13030', '16632', '81007'], ['20047', '11110', '22501', '39221', '18008', '7001', '81007'], ['25163', '9811', '19640', '18008', '7001', '13037'], ['8400', '25103', '9104', '7001', '19640', '18008', '11110'], ['36005', '20071', '7001', '18008', '19611', '13014', '81007', '81007'], ['28925', '20071', '11110', '7001', '18008', '39221', '81007'], ['36306', '20101', '4003', '17440', '28504', '11081', '7001', '19640', '18008', '81007', '81007'], ['28900', '7001', '19624', '18008'], ['6210', '20030', '7001', '23525', '18008'], ['26034', '25606', '11110', '7001', '23525', '18008'], ['25902', '28916', '76057', '19640', '13039', '7100'], ['9811', '13039', '19700', '76022', '11114'], ['20009', '1007', '25902', '9811', '11114', '12114', '18009', '81061'], ['25902', '25103', '9104', '12114', '18008', '7001', '81029', '81009'], ['25906', '8400', '25606', '11110', '19700', '7001', '18008', '13037'], ['9811', '11107', '81022'], ['76061', '13034', '20051', '

In [5]:
#ADAPTATION EXERSYS 
#Créer des jeux de training, validation, set

import random
import pickle
data=REP_vraisalimfreqliste

# # Define the proportions for train, validation, and test datasets (as percentages)
# train_ratio = 0.7
# validation_ratio = 0.15
# test_ratio = 0.15

# # Calculate the number of samples for each dataset
# num_samples = len(data)
# num_train = int(num_samples * train_ratio)
# num_validation = int(num_samples * validation_ratio)
# num_test = num_samples - num_train - num_validation

# # Shuffle the data randomly
# random.shuffle(data)

# # Split the data into three datasets: train, validation, and test
# train_data = data[:num_train]
# validation_data = data[num_train:num_train + num_validation]
# test_data = data[num_train + num_validation:]


# # Save the train_data using pickle
# with open('train_data.pkl', 'wb') as f:
#     pickle.dump(train_data, f)
    
# # Save the validation_data  using pickle
# with open('validation_data.pkl', 'wb') as g:
#     pickle.dump(validation_data, g)

# # Save the test_data using pickle
# with open('test_data.pkl', 'wb') as h:
#     pickle.dump(test_data, h)
    
# Load the train_data from the saved file
with open('train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

# Load the validation_data from the saved file
with open('validation_data.pkl', 'rb') as g:
    validation_data = pickle.load(g)

# Load the test_data from the saved file
with open('test_data.pkl', 'rb') as h:
    test_data = pickle.load(h)

# Now, 'train_data', 'validation_data', and 'test_data' contain the respective datasets
print("Train Data:", train_data[:20])
print("Validation Data:", validation_data[:20])
print("Test Data:", test_data[:20])

# Convertir les sous-listes en phrases (chaînes de caractères)
train_data_sentences = [' '.join(sublist) for sublist in train_data]
validation_data_sentences = [' '.join(sublist) for sublist in validation_data]
test_data_sentences = [' '.join(sublist) for sublist in test_data]
print(train_data_sentences[:2])

all_sentences = []
all_sentences.extend(train_data_sentences)
all_sentences.extend(validation_data_sentences)
all_sentences.extend(test_data_sentences)
print(len(all_sentences))

Train Data: [['5206', '20047', '12115', '13034', '7001', '11017', '81063'], ['76020'], ['25405', '36005', '4018', '23490', '18066', '5206', '18004'], ['25410', '28916', '18066', '12100', '22010', '13008'], ['9811', '12114', '18066', '18017'], ['6255', '20030', '5210', '18008'], ['20009', '6999', '9822', '13005', '76056', '81024', '81008', '81063'], ['9811', '28504', '19402', '13024', '18066', '81009'], ['9811', '12020', '13014', '31016', '18008'], ['13037', '19760', '76020'], ['30742', '4010', '90068', '23477', '7001', '18066', '18004'], ['9811', '6255', '11107', '20035', '5001', '5205', '7110', '81007', '81007'], ['4003', '20030', '17270', '26025', '5205', '90068', '13039', '7001', '18004', '81052', '81022', '81007', '81006', '81007', '81006'], ['25002', '9104', '7001', '19628', '81024', '81009'], ['18066', '6200', '4018', '90068', '23585', '7001'], ['26015', '17440', '25031', '13044', '5205', '18008', '7001', '18004', '31016', '81022'], ['25902', '19606', '18008', '81052'], ['1006', 

In [6]:
# Aplatir la liste de listes
aplatie = [item for sublist in REP_vraisalimfreqliste for item in sublist]
# Obtenir les valeurs uniques
features = list(set(aplatie))

print(len(features))
print(features)

335
['28802', '13034', '4010', '81011', '19609', '9822', '11161', '25914', '6100', '23684', '34002', '18073', '18017', '81061', '23490', '25018', '12999', '81038', '13040', '31008', '81007', '90110', '23000', '18071', '20040', '6520', '81022', '25410', '36306', '13002', '13043', '18060', '16402', '20172', '22010', '20008', '11109', '19590', '39212', '92031', '4005', '12747', '76016', '11107', '20068', '22014', '28916', '19730', '26039', '19640', '30155', '8551', '18018', '5204', '8704', '6999', '5210', '21503', '20071', '18008', '36005', '8400', '20076', '20099', '25420', '25515', '1005', '39200', '11015', '4006', '76057', '26025', '30780', '23585', '11164', '25002', '13000', '9081', '5207', '26140', '39511', '12010', '20036', '12803', '13033', '30005', '19600', '22500', '12045', '20014', '90057', '11110', '7001', '19501', '20011', '25947', '13044', '28101', '9104', '76020', '81063', '11004', '8800', '2007', '13009', '26034', '30742', '13134', '1006', '31016', '81009', '25945', '17130'

# B. Preprocessing

In [7]:
#ADAPTATION EXERSYS 
class Preprocessor:
    def process_batch(self, batch: List[str]):            
        return pad_sequence([self.string2idx(text) for text in batch])

class SentencePiecePreprocessor(Preprocessor):
    """Tokenizer Sentence Piece"""
    def embeddings(self, dimension: int):
        return nn.Embedding(self.vocab_size, dimension, padding_idx=self.pad_index)
    
    @property
    def vocab_size(self):
        return self.spm_tokenizer.vocab_size()

    def __init__(self, name: str, sentences: Iterable[str], vocab_size=len(features)+4, force=False):
        modelpath = BASEPATH / f"{name}-{vocab_size}.model"
        if not modelpath.exists() or force:
            print(f"Entraînement de SPM, sortie {BASEPATH}/{name}-{vocab_size}", flush=True)
            spm.SentencePieceTrainer.train(
                sentence_iterator=iter(sentences), 
                model_prefix=f"{BASEPATH}/{name}-{vocab_size}", 
                vocab_size=vocab_size,
                model_type="word",
                pad_id=0,                
                unk_id=1,
                bos_id=2,
                eos_id=3
            )
            
        self.spm_tokenizer = spm.SentencePieceProcessor()
        self.spm_tokenizer.load(str(modelpath))

        self.pad_index = self.spm_tokenizer.pad_id()
        self.eos_index = self.spm_tokenizer.eos_id()
        self.bos_index = self.spm_tokenizer.bos_id()
        self.oov_index = None

        self.spm_tokenizer.SetEncodeExtraOptions("bos:eos")

    def string2idx(self, s: str) -> torch.Tensor:
        return torch.tensor(self.spm_tokenizer.EncodeAsIds(s))

    def tokenizer(self, x):
        """Segmentation du texte en sous-mots"""
        return self.spm_tokenizer.encode_as_pieces(x)

    def id2token(self, ix):
        return self.spm_tokenizer.IdToPiece(ix)

    def decode(self, ids: List[int]):
        return self.spm_tokenizer.Decode(ids)


# Si vous voulez essayer avec des mots
# preprocessor = FastTextProcessor()

preprocessor = SentencePiecePreprocessor("spm", all_sentences, force=False)
print(f"Vocabulary size: {preprocessor.vocab_size}")

Entraînement de SPM, sortie xp\generation/spm-339
Vocabulary size: 339


In [8]:
#ADAPTATION EXERSYS 
# exemple de décomposition 
preprocessor.tokenizer(['25902 25019 13039 5210 13005 18066'])

[['<s>', '▁25902', '▁25019', '▁13039', '▁5210', '▁13005', '▁18066', '</s>']]

In [9]:
#ADAPTATION EXERSYS 
out = preprocessor.process_batch(['25902 25019 13039 5210 13005 18066']) # => tenseur d'indices présenté en colonne
print(out)

tensor([[  2],
        [ 17],
        [113],
        [ 11],
        [ 51],
        [ 32],
        [  5],
        [  3]])


In [10]:
#ADAPTATION EXERSYS 
# transformation inverse (index => token)
# attention (toujours aux dimensions) : si on oublie le squeeze => list de token

print(preprocessor.decode(out.tolist()))
print(preprocessor.decode(out.squeeze().tolist()))

['', '25902', '25019', '13039', '5210', '13005', '18066', '']
25902 25019 13039 5210 13005 18066


In [12]:
#ADAPTATION EXERSYS 
# reflexion sur le padding
x_exemple = preprocessor.process_batch(['25902 25019 13039 5210 13005 18066', '28925 20066 20047 11110 12115 9811 9811 9811'])

print("Dimension de x :", x_exemple.size()) # T x batch_size x dim_input
print(x_exemple)

Dimension de x : torch.Size([10, 2])
tensor([[  2,   2],
        [ 17, 180],
        [113, 158],
        [ 11,  14],
        [ 51,  18],
        [ 32, 138],
        [  5,   7],
        [  3,   7],
        [  0,   7],
        [  0,   3]])


In [13]:
#ADAPTATION EXERSYS 
# récupération automatique d'une lookup table (tableau d'embeddings):
embeddings = preprocessor.embeddings(50) 

print("taille de la table:", embeddings.num_embeddings, "x", embeddings.embedding_dim)

taille de la table: 339 x 50


In [14]:
#ADAPTATION EXERSYS 
# verification sur la nature de la représentation du mot de padding

x = embeddings(preprocessor.process_batch(['25902 25019 13039 5210 13005 18066', '28925 20066 20047 11110 12115 9811 9811 9811']))
print("Dimension de x :", x.size()) # T x batch_size x dim_input
print("mot :", x[9,0,:].data) # 9e mot du doc 0 (toutes les dimensions de la représentation)

Dimension de x : torch.Size([10, 2, 50])
mot : tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])


In [9]:
# avant entrainement du modèle
z = embeddings(preprocessor.process_batch(['25902']))
print("Dimension de z :", z.size())
print(z)

Dimension de z : torch.Size([3, 1, 50])
tensor([[[-2.0201e-01,  1.8031e+00,  3.8676e-01,  1.1606e+00,  6.4701e-01,
          -6.7094e-01,  4.8561e-01,  2.9334e-01,  3.2470e-01,  4.1586e-01,
          -9.8234e-01, -2.3510e-01,  2.0420e-01,  9.3792e-01, -9.7723e-01,
          -7.0509e-01,  1.0098e+00,  4.6374e-01, -5.9378e-01,  9.3944e-01,
          -1.5976e+00,  2.3286e-01,  1.2853e+00, -2.3156e-01,  1.9479e+00,
           9.2192e-01, -2.7252e-01,  1.8955e+00, -7.2658e-01,  5.9496e-01,
          -1.2388e+00, -1.9278e+00, -1.1768e-01, -2.0479e-01, -2.4322e+00,
          -2.1152e+00, -9.4308e-01,  1.2231e+00,  1.6310e+00,  1.3477e+00,
           1.2446e+00, -2.9931e-01,  5.6959e-01,  3.1627e-01,  2.7398e+00,
          -1.2192e+00,  2.4343e-02,  6.1029e-01, -1.3354e-01,  2.0118e-02]],

        [[-1.0855e+00,  1.2693e-01,  2.7013e+00, -3.3852e-02,  8.9728e-01,
          -1.1368e+00, -8.1332e-01,  8.4707e-01,  6.0079e-01, -3.8856e-01,
           4.2777e-01, -3.1189e-01, -5.0465e-01,  5.6208e

In [15]:
#Vérifier la bonne tokenisation de tous les aliments
rand_vec=0
for token_id in range(preprocessor.vocab_size):
    token = preprocessor.id2token(token_id)
    print(token)
    if token.startswith("▁"):
        if not str(token[1:]) in features:
            rand_vec+=1
            print("token not in model:", token) 


<pad>
<unk>
<s>
</s>
▁7001
▁18066
▁18008
▁9811
▁18004
▁25604
▁81024
▁13039
▁81008
▁28916
▁20047
▁81009
▁31016
▁25902
▁11110
▁16400
▁17270
▁9104
▁90068
▁81022
▁6255
▁12114
▁81052
▁11018
▁5205
▁19600
▁81007
▁5203
▁13005
▁13024
▁12999
▁36005
▁4003
▁6200
▁18018
▁81025
▁7100
▁76020
▁7012
▁19628
▁22010
▁11017
▁20009
▁20062
▁4027
▁19624
▁22500
▁5210
▁4010
▁25903
▁13037
▁20030
▁81011
▁4005
▁81010
▁81006
▁25516
▁28101
▁17440
▁13034
▁11108
▁18044
▁11107
▁20026
▁81061
▁13026
▁28301
▁81029
▁11054
▁36306
▁20031
▁76056
▁19402
▁8211
▁5206
▁76016
▁30300
▁20035
▁20210
▁92044
▁4019
▁11013
▁20008
▁26030
▁4018
▁26039
▁26140
▁19800
▁92024
▁13044
▁28504
▁11015
▁25018
▁76057
▁18071
▁25405
▁18009
▁12500
▁28800
▁4006
▁20021
▁81042
▁19606
▁20003
▁17001
▁19640
▁19765
▁13043
▁20071
▁25019
▁36004
▁81038
▁12001
▁20011
▁20027
▁11109
▁6210
▁19501
▁22501
▁25127
▁11000
▁19510
▁30780
▁20099
▁23499
▁13014
▁25608
▁16402
▁5001
▁36018
▁11014
▁23490
▁25002
▁81012
▁12115
▁20017
▁20036
▁25056
▁17700
▁39200
▁13030
▁20045
▁81037

In [16]:
#detecter les erreurs de tokenisation par sentencepiece
import copy
features_miss=copy.deepcopy(features)
for token_id in range(preprocessor.vocab_size):
    token = preprocessor.id2token(token_id)
    if token.startswith("▁"):
        features_miss.remove(str(token[1:]))
print(features_miss)

[]


# C. RNN

In [17]:
#ADAPTATION EXERSYS 
TRAIN_BATCHSIZE = 100
VALID_BATCHSIZE = 20

def train_1to1(preprocessor, model, epochs, all_sentences):
    """Fonction d'entraînement (teacher forcing)"""
    print(f"Training {model.name}")
    
    # On nettoie le rep. de log
    shutil.rmtree(f"{TB_PATH}/{model.name}", ignore_errors = True)
    writer = SummaryWriter(f"{TB_PATH}/{model.name}")
    
    optim = torch.optim.Adam(model.parameters(), lr=1e-3)
    model = model.to(device)

    train_loader = DataLoader(all_sentences[:len(train_data_sentences)], TRAIN_BATCHSIZE, shuffle=True)
    test_loader = DataLoader(all_sentences[len(train_data_sentences):(len(train_data_sentences)+len(validation_data_sentences))], VALID_BATCHSIZE, shuffle=False)
    loss = nn.CrossEntropyLoss(ignore_index=preprocessor.pad_index)  # classification avec une classe qui ne retourne pas d'erreur
    
    for epoch in tqdm(range(epochs)):
        cumloss, count =  0, 0
        model.train()
        for batch in train_loader:
            optim.zero_grad()
            x = preprocessor.process_batch(batch).to(device)
        
            # Mode "Teacher forcing"
            x, y = x[:-1], x[1:] # cf schéma ci-dessus

            yhat, hidden = model.forward(x)
            l = loss(yhat.view(-1,yhat.size(2)), y.view(-1))
            l.backward()
            optim.step()
            cumloss += l*len(x)
            count += len(x)
        writer.add_scalar('loss/train', cumloss/count, epoch)

        if epoch % 1 == 0:
            model.eval()
            with torch.no_grad():
                cumloss, count = 0, 0
                for batch in test_loader:                  
                    x = preprocessor.process_batch(batch)
                    x, y = x[:-1].to(device), x[1:].to(device)
                    yhat, hidden = model(x)
                    cumloss += loss(yhat.view(-1,yhat.size(2)),y.view(-1))*len(x)
                    count += len(x)
                writer.add_scalar(f'loss/test',cumloss/count,epoch)

                


class RNN(nn.Module):
    def __init__(self, embeddings, hidden_dim):
        super().__init__()
        self.name = f"rnn-{embeddings.embedding_dim}-{hidden_dim}"
        self.embeddings = embeddings # stockage des embeddings (fournis par le preprocessor)
        self.rnn = nn.RNN(embeddings.embedding_dim, hidden_dim, nonlinearity="tanh")
        self.fc = nn.Linear(hidden_dim, embeddings.num_embeddings)

        
    
    def forward(self, x, h_0=None): 
        x = self.embeddings(x)
        output, hidden = self.rnn(x, h_0) # reflexion sur la dimension de output
        return self.fc(output), hidden
        # Attention de bien prendre en compte h_0 dans l'appel du RRN
        

# On récupère des embeddings pour le modèle
# embeddings = preprocessor.embeddings(50)

# Création du RNN avec 100 états cachés
rnn_model = RNN(embeddings, 100)

# Entraînement du modèle:
train_1to1(preprocessor, rnn_model, 22, all_sentences)

Training rnn-50-100


100%|██████████| 22/22 [03:44<00:00, 10.18s/it]


In [13]:
print(len(train_data_sentences))
print(len(validation_data_sentences))
print(len(test_data_sentences))

8096
1735
1736


In [14]:
# après entrainement du modèle
z_new = embeddings(preprocessor.process_batch(['25902']))
print("Dimension de z :", z_new.size())
print(z_new)

Dimension de z : torch.Size([3, 1, 50])
tensor([[[-1.0921e-01,  1.9647e+00,  4.2780e-01,  1.1617e+00,  6.9006e-01,
          -7.0610e-01,  4.7020e-01,  3.2710e-01,  2.5717e-01,  5.2150e-01,
          -1.0451e+00, -1.9439e-01,  2.1235e-01,  1.0643e+00, -1.0411e+00,
          -6.8591e-01,  1.0259e+00,  4.9029e-01, -5.1147e-01,  9.8898e-01,
          -1.6559e+00,  2.0173e-01,  1.3715e+00, -1.0978e-01,  2.0575e+00,
           8.7288e-01, -2.9756e-01,  2.0556e+00, -7.0388e-01,  7.3186e-01,
          -1.4244e+00, -2.0626e+00, -1.9656e-01, -1.1097e-01, -2.5237e+00,
          -2.2038e+00, -1.0355e+00,  1.2942e+00,  1.6383e+00,  1.4370e+00,
           1.3489e+00, -3.8611e-01,  5.8388e-01,  2.4124e-01,  2.8007e+00,
          -1.2367e+00, -1.4910e-01,  8.1302e-01, -1.6351e-01,  7.8985e-02]],

        [[-1.1721e+00,  3.4121e-01,  2.7749e+00, -4.7502e-01,  1.1848e+00,
          -1.7925e+00, -1.0345e+00,  9.9546e-01,  1.1078e+00, -3.3109e-01,
           1.9068e-01, -2.8206e-01, -2.5116e-01,  2.1786e

In [19]:
import pandas as pd

df_test_10_100=pd.read_csv('./other/rnn-500ep_10dim-emb_100dim-hid_test_loss.csv', names=['time', 'step', 'value'], header=1)
df_test_10_100.head()
argmin_index_10_100=df_test_10_100["value"].idxmin()
argmin_loss_10_100=df_test_10_100.loc[argmin_index_10_100, "value"]
argmin_loss_epoch_10_100=df_test_10_100.loc[argmin_index_10_100, "step"]
print("loss mini:", round(argmin_loss_10_100, 2), "epoch loss mini:", argmin_loss_epoch_10_100)

loss mini: 3.93 epoch loss mini: 41


In [20]:
df_test_50_100=pd.read_csv('./other/rnn-500ep_50dim-emb_100dim-hid_test_loss.csv', names=['time', 'step', 'value'],  header=1)
df_test_50_100.head()
argmin_index_50_100=df_test_50_100["value"].idxmin()
argmin_loss_50_100=df_test_50_100.loc[argmin_index_50_100, "value"]
argmin_loss_epoch_50_100=df_test_50_100.loc[argmin_index_50_100, "step"]
argmin_loss_epoch_50_100=df_test_50_100.loc[argmin_index_50_100, "step"]
print("loss mini:", round(argmin_loss_50_100, 2), "epoch loss mini:", argmin_loss_epoch_50_100)

loss mini: 3.9 epoch loss mini: 22


In [21]:
df_test_10_50=pd.read_csv('./other/rnn-500ep_10dim-emb_50dim-hid_test_loss.csv', names=['time', 'step', 'value'], header=1)
df_test_10_50.head()
argmin_index_10_50=df_test_10_50["value"].idxmin()
argmin_loss_10_50=df_test_10_50.loc[argmin_index_10_50, "value"]
argmin_loss_epoch_10_50=df_test_10_50.loc[argmin_index_10_50, "step"]
argmin_loss_epoch_10_50=df_test_10_50.loc[argmin_index_10_50, "step"]
print("loss mini:", round(argmin_loss_10_50, 2), "epoch loss mini:", argmin_loss_epoch_10_50)

loss mini: 3.93 epoch loss mini: 88


In [22]:
#evaluer performance sur le set de test: métrique loss crossentropie
TEST_BATCHSIZE = len(all_sentences[(len(train_data_sentences)+len(validation_data_sentences)):])

def assess_ce(preprocessor, model, all_sentences):    
    model = model.to(device)
    realtest_loader = DataLoader(all_sentences[(len(train_data_sentences)+len(validation_data_sentences)):], TEST_BATCHSIZE, shuffle=False)
    loss = nn.CrossEntropyLoss(ignore_index=preprocessor.pad_index)  # classification avec une classe qui ne retourne pas d'erreur
    model.eval()
    with torch.no_grad():
        cumloss, count = 0, 0
        for batch in realtest_loader:
            x = preprocessor.process_batch(batch)
            x, y = x[:-1].to(device), x[1:].to(device)
            yhat, hidden = model(x)
            cumloss += loss(yhat.view(-1,yhat.size(2)),y.view(-1))*len(x)
            count += len(x)
    return("cumloss/count:", cumloss/count)

assess_ce(preprocessor, rnn_model, all_sentences)

('cumloss/count:', tensor(3.9445))

In [23]:
#evaluer performance sur le set de test: métrique de classification 

TEST_BATCHSIZE=1

def assess_classif(preprocessor, model, all_sentences):    
    count=0
    alim_bien_pred=[]
    percent_ok=0
    realtest_loader = DataLoader(all_sentences[(len(train_data_sentences)+len(validation_data_sentences)):], TEST_BATCHSIZE, shuffle=False)
    model.eval()
    with torch.no_grad():
        for batch in realtest_loader:
            x = preprocessor.process_batch(batch)
#             print("x", x)
            x, y = x[:-1].to(device), x[1:].to(device)
            yhat, hidden = model(x)
#             print("x", x)
#             print("y size", y.shape)
#             print("y", y)
#             print("yhat size", yhat.shape)
#             print("yhat", yhat)
#             print("nombre d'aliments", yhat.shape[0])
            predicted_classes = torch.argmax(yhat, dim=2)
#             print("predicted classes", predicted_classes)
            num_equal_elements = (predicted_classes == y).sum().item()
            alim_bien_pred.append(y[predicted_classes == y].tolist())
            print("% d'aliments bien classés pour 1 repas", round(num_equal_elements/yhat.shape[0], 2))
            count += yhat.shape[0]
            percent_ok += num_equal_elements
        print("counts", count)
        print("nok", percent_ok)
        print(alim_bien_pred)
        return("pourcentage global de bonne classification:", round(percent_ok/count, 3))

        

assess_classif(preprocessor, rnn_model, all_sentences)


% d'aliments bien classés pour 1 repas 0.14
% d'aliments bien classés pour 1 repas 0.12
% d'aliments bien classés pour 1 repas 0.14
% d'aliments bien classés pour 1 repas 0.22
% d'aliments bien classés pour 1 repas 0.17
% d'aliments bien classés pour 1 repas 0.17
% d'aliments bien classés pour 1 repas 0.12
% d'aliments bien classés pour 1 repas 0.17
% d'aliments bien classés pour 1 repas 0.17
% d'aliments bien classés pour 1 repas 0.0
% d'aliments bien classés pour 1 repas 0.0
% d'aliments bien classés pour 1 repas 0.14
% d'aliments bien classés pour 1 repas 0.25
% d'aliments bien classés pour 1 repas 0.22
% d'aliments bien classés pour 1 repas 0.5
% d'aliments bien classés pour 1 repas 0.2
% d'aliments bien classés pour 1 repas 0.2
% d'aliments bien classés pour 1 repas 0.12
% d'aliments bien classés pour 1 repas 0.14
% d'aliments bien classés pour 1 repas 0.2
% d'aliments bien classés pour 1 repas 0.0
% d'aliments bien classés pour 1 repas 0.38
% d'aliments bien classés pour 1 repas 

% d'aliments bien classés pour 1 repas 0.5
% d'aliments bien classés pour 1 repas 0.6
% d'aliments bien classés pour 1 repas 0.2
% d'aliments bien classés pour 1 repas 0.0
% d'aliments bien classés pour 1 repas 0.29
% d'aliments bien classés pour 1 repas 0.5
% d'aliments bien classés pour 1 repas 0.1
% d'aliments bien classés pour 1 repas 0.0
% d'aliments bien classés pour 1 repas 0.25
% d'aliments bien classés pour 1 repas 0.17
% d'aliments bien classés pour 1 repas 0.11
% d'aliments bien classés pour 1 repas 0.14
% d'aliments bien classés pour 1 repas 0.12
% d'aliments bien classés pour 1 repas 0.0
% d'aliments bien classés pour 1 repas 0.22
% d'aliments bien classés pour 1 repas 0.67
% d'aliments bien classés pour 1 repas 0.0
% d'aliments bien classés pour 1 repas 0.29
% d'aliments bien classés pour 1 repas 0.17
% d'aliments bien classés pour 1 repas 0.43
% d'aliments bien classés pour 1 repas 0.25
% d'aliments bien classés pour 1 repas 0.17
% d'aliments bien classés pour 1 repas 0.

% d'aliments bien classés pour 1 repas 0.27
% d'aliments bien classés pour 1 repas 0.4
% d'aliments bien classés pour 1 repas 0.2
% d'aliments bien classés pour 1 repas 0.29
% d'aliments bien classés pour 1 repas 0.29
% d'aliments bien classés pour 1 repas 0.18
% d'aliments bien classés pour 1 repas 0.14
% d'aliments bien classés pour 1 repas 0.2
% d'aliments bien classés pour 1 repas 0.29
% d'aliments bien classés pour 1 repas 0.2
% d'aliments bien classés pour 1 repas 0.0
% d'aliments bien classés pour 1 repas 0.12
% d'aliments bien classés pour 1 repas 0.33
% d'aliments bien classés pour 1 repas 0.25
% d'aliments bien classés pour 1 repas 0.2
% d'aliments bien classés pour 1 repas 0.6
% d'aliments bien classés pour 1 repas 0.4
% d'aliments bien classés pour 1 repas 0.12
% d'aliments bien classés pour 1 repas 0.33
% d'aliments bien classés pour 1 repas 0.22
% d'aliments bien classés pour 1 repas 0.14
% d'aliments bien classés pour 1 repas 0.25
% d'aliments bien classés pour 1 repas 0

% d'aliments bien classés pour 1 repas 0.1
% d'aliments bien classés pour 1 repas 0.2
% d'aliments bien classés pour 1 repas 0.0
% d'aliments bien classés pour 1 repas 0.33
% d'aliments bien classés pour 1 repas 0.18
% d'aliments bien classés pour 1 repas 0.43
% d'aliments bien classés pour 1 repas 0.43
% d'aliments bien classés pour 1 repas 0.12
% d'aliments bien classés pour 1 repas 0.2
% d'aliments bien classés pour 1 repas 0.15
% d'aliments bien classés pour 1 repas 0.17
% d'aliments bien classés pour 1 repas 0.33
% d'aliments bien classés pour 1 repas 0.2
% d'aliments bien classés pour 1 repas 0.14
% d'aliments bien classés pour 1 repas 0.2
% d'aliments bien classés pour 1 repas 0.14
% d'aliments bien classés pour 1 repas 0.18
% d'aliments bien classés pour 1 repas 0.29
% d'aliments bien classés pour 1 repas 0.2
% d'aliments bien classés pour 1 repas 0.17
% d'aliments bien classés pour 1 repas 0.0
% d'aliments bien classés pour 1 repas 0.33
% d'aliments bien classés pour 1 repas 0

('pourcentage global de bonne classification:', 0.203)

In [25]:
with open("./dic_alim.pkl", "rb") as fichier:
    dic_alim = pickle.load(fichier)

In [27]:
#evaluer performance sur le set de test: métrique de classification SANS LE EOS 

from collections import Counter
TEST_BATCHSIZE=1

def assess_classif(preprocessor, model, all_sentences):    
    count=0
    percent_ok=0
    alim_bien_pred=[]
    alim_liste=[]
    realtest_loader = DataLoader(all_sentences[(len(train_data_sentences)+len(validation_data_sentences)):], TEST_BATCHSIZE, shuffle=False)
    model.eval()
    with torch.no_grad():
        for batch in realtest_loader:
            x = preprocessor.process_batch(batch)
#             print("x", x)
            x_old, y_old = x[:-1].to(device), x[1:].to(device)
            yhat_old, hidden = model(x_old)
            x, y = x_old[:-1], y_old[:-1]
            yhat = yhat_old[:-1, :, :]
            print("x", x)
#             print("y size", y.shape)
#             print("y", y)
#             print("yhat_old size", yhat_old.shape)
#             print("yhat size", yhat.shape)
#             print("yhat", yhat)
#             print("nombre d'aliments", yhat.shape[0])
            predicted_classes = torch.argmax(yhat, dim=2)
#             print("predicted classes", predicted_classes)
            num_equal_elements = (predicted_classes == y).sum().item()
            alim_bien_pred.append(y[predicted_classes == y].tolist())            
#             print("% d'aliments bien classés pour 1 repas", round(num_equal_elements/yhat.shape[0], 2))
            count += yhat.shape[0]
            percent_ok += num_equal_elements
        print("counts", count)
        print("nok", percent_ok)
        return("pourcentage global de bonne classification sans le EOS:", round(percent_ok/count, 3))

assess_classif(preprocessor, rnn_model, all_sentences)


x tensor([[  2],
        [145],
        [ 36],
        [ 94],
        [ 50],
        [198]])
x tensor([[  2],
        [  7],
        [223],
        [ 40],
        [101],
        [206],
        [  5]])
x tensor([[  2],
        [ 44],
        [  4],
        [ 19],
        [237],
        [ 13]])
x tensor([[  2],
        [122],
        [118],
        [ 76],
        [  4],
        [  5],
        [ 63],
        [ 12]])
x tensor([[  2],
        [136],
        [  4],
        [  6],
        [ 54]])
x tensor([[  2],
        [161],
        [ 72],
        [233],
        [278]])
x tensor([[  2],
        [130],
        [ 70],
        [207],
        [109],
        [ 81],
        [ 23]])
x tensor([[  2],
        [212],
        [238],
        [ 38],
        [  8]])
x tensor([[  2],
        [184],
        [ 57],
        [  6],
        [276]])
x tensor([[ 2],
        [47],
        [24]])
x tensor([[  2],
        [  5],
        [193],
        [164],
        [ 21]])
x tensor([[  2],
        [  6],
        

x tensor([[  2],
        [  7],
        [ 19],
        [261],
        [ 28]])
x tensor([[  2],
        [  4],
        [  5],
        [118],
        [248],
        [  7],
        [  9],
        [121],
        [ 33],
        [ 76],
        [ 26],
        [ 10],
        [ 30],
        [ 30]])
x tensor([[ 2],
        [53],
        [92]])
x tensor([[  2],
        [139],
        [197],
        [117]])
x tensor([[  2],
        [169],
        [ 55],
        [ 22],
        [  5],
        [ 44],
        [146]])
x tensor([[  2],
        [ 61],
        [ 96],
        [ 21],
        [109],
        [ 63],
        [  5]])
x tensor([[  2],
        [ 34],
        [ 77],
        [232],
        [ 67],
        [  6],
        [106],
        [  4],
        [  8]])
x tensor([[  2],
        [  7],
        [ 25],
        [155],
        [  5]])
x tensor([[  2],
        [ 82],
        [119],
        [220],
        [ 20],
        [ 21],
        [ 66],
        [ 91],
        [  4],
        [  5]])
x tensor([[  2],

x tensor([[  2],
        [202]])
x tensor([[ 2],
        [24],
        [57],
        [ 4],
        [ 5]])
x tensor([[ 2],
        [99],
        [79]])
x tensor([[  2],
        [ 21],
        [171],
        [  6],
        [ 66]])
x tensor([[  2],
        [ 70],
        [ 48],
        [  4],
        [ 34],
        [245],
        [  6]])
x tensor([[  2],
        [228]])
x tensor([[  2],
        [102],
        [ 19],
        [ 84],
        [ 37],
        [  4],
        [ 28],
        [215],
        [  8],
        [ 16],
        [ 30]])
x tensor([[  2],
        [123],
        [320],
        [ 41],
        [338],
        [  8]])
x tensor([[  2],
        [ 14],
        [ 45],
        [ 24],
        [ 20],
        [125],
        [237],
        [  5]])
x tensor([[  2],
        [114],
        [ 47],
        [ 49],
        [  8],
        [105],
        [ 12]])
x tensor([[  2],
        [ 13],
        [ 50],
        [ 20],
        [178],
        [  5]])
x tensor([[  2],
        [ 13],
        [314]

x tensor([[  2],
        [ 35],
        [293],
        [ 14],
        [152],
        [  6],
        [219],
        [  4]])
x tensor([[  2],
        [204]])
x tensor([[  2],
        [ 54],
        [103],
        [139],
        [134],
        [ 81],
        [269],
        [ 20],
        [ 22],
        [ 40],
        [100]])
x tensor([[  2],
        [ 46],
        [142],
        [ 40],
        [310],
        [227],
        [ 19],
        [153],
        [103],
        [208],
        [ 63],
        [100],
        [ 28]])
x tensor([[  2],
        [  4],
        [ 28],
        [ 99],
        [127],
        [ 18],
        [208]])
x tensor([[  2],
        [  9],
        [ 20],
        [261],
        [ 13],
        [241],
        [111],
        [  5],
        [  4]])
x tensor([[  2],
        [234],
        [  6],
        [ 29],
        [ 37]])
x tensor([[ 2],
        [17],
        [25]])
x tensor([[ 2],
        [ 5],
        [50],
        [29]])
x tensor([[  2],
        [157],
        [ 87],
   

x tensor([[  2],
        [158],
        [195],
        [244],
        [ 14],
        [168],
        [111],
        [ 64]])
x tensor([[2],
        [7]])
x tensor([[ 2],
        [46],
        [70]])
x tensor([[  2],
        [141],
        [150],
        [  4]])
x tensor([[  2],
        [167],
        [ 44],
        [ 92],
        [ 42]])
x tensor([[ 2],
        [44],
        [14],
        [72],
        [37],
        [52],
        [ 5]])
x tensor([[  2],
        [ 36],
        [  9],
        [ 19],
        [128]])
x tensor([[ 2],
        [ 4],
        [72]])
x tensor([[2]])
x tensor([[ 2],
        [60],
        [ 5],
        [ 8]])
x tensor([[ 2],
        [52],
        [87]])
x tensor([[  2],
        [  9],
        [233]])
x tensor([[  2],
        [183]])
x tensor([[2]])
x tensor([[  2],
        [ 24],
        [101]])
x tensor([[  2],
        [123],
        [101],
        [160],
        [144],
        [  5]])
x tensor([[  2],
        [ 14],
        [158],
        [ 13],
        [310],
   

('pourcentage global de bonne classification sans le EOS:', 0.099)

In [28]:
#evaluer performance sur le set de test: métrique de classification TOP 3

TEST_BATCHSIZE=1

def assess_classif_top3(preprocessor, model, all_sentences):    
    count=0
    percent_ok=0
    realtest_loader = DataLoader(all_sentences[(len(train_data_sentences)+len(validation_data_sentences)):], TEST_BATCHSIZE, shuffle=False)
    model.eval()
    with torch.no_grad():
        for batch in realtest_loader:
            num_top3=0
            x = preprocessor.process_batch(batch)
#             print("x", x)
            x, y = x[:-1].to(device), x[1:].to(device)
            yhat, hidden = model(x)
#             print("x", x)
#             print("y", y)
#             print("y size", y.shape)
#             print("yhat size", yhat.shape)
#             print("yhat", yhat)
#             print("nombre d'aliments", yhat.shape[0])
            top_values, top_indices = torch.topk(yhat, k=3, dim=2)
#             print("top3 classes size", top_indices.shape)            
#             print("top3 classes", top_indices)
            for i in range(yhat.shape[0]):
                if y[i,0] in top_indices[i,0,:]:
                    num_top3 +=1
            print("% d'aliments dans le top3 pour 1 repas", round(num_top3/yhat.shape[0], 2))
            count += yhat.shape[0]
            percent_ok += num_top3
        return("pourcentage global de bonne classification:", round(percent_ok/count, 2))

assess_classif_top3(preprocessor, rnn_model, all_sentences)

% d'aliments dans le top3 pour 1 repas 0.14
% d'aliments dans le top3 pour 1 repas 0.38
% d'aliments dans le top3 pour 1 repas 0.14
% d'aliments dans le top3 pour 1 repas 0.56
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.12
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.25
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.29
% d'aliments dans le top3 pour 1 repas 0.25
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.4
% d'aliments dans le top3 pour 1 repas 0.6
% d'aliments dans le top3 pour 1 repas 0.62
% d'aliments dans le top3 pour 1 repas 0.29
% d'aliments dans le top3 pour 1 repas 0.2
% d'aliments dans le top3 pour 1 repas 0.67
% d'aliments dans le top3 pour 1 repas 0.62
% d'aliments dans le top3 pour 1 repas

% d'aliments dans le top3 pour 1 repas 0.25
% d'aliments dans le top3 pour 1 repas 0.4
% d'aliments dans le top3 pour 1 repas 0.4
% d'aliments dans le top3 pour 1 repas 0.25
% d'aliments dans le top3 pour 1 repas 0.12
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.2
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.3
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.67
% d'aliments dans le top3 pour 1 repas 0.14
% d'aliments dans le top3 pour 1 repas 0.27
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.2
% d'aliments dans le top3 pour 1 repas 0.3
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.38
% d'aliments dans le top3 pour 1 repas 0.17
% d'aliments dans le top3 pour 1 repas 0.31

% d'aliments dans le top3 pour 1 repas 0.17
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.25
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.57
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.25
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.14
% d'aliments dans le top3 pour 1 repas 0.57
% d'aliments dans le top3 pour 1 repas 0.14
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.2
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.29
% d'aliments dans le top3 pour 1 repas 0.4
% d'aliments dans le top3 pour 1 repas 0.67
% d'aliments dans le top3 pour 1 repas 0.38
% d'aliments dans le top3 pour 1 repas 0.11
% d'aliments dans le top3 pour 1 repas 0.25
% d'aliments dans le top3 pour 1 repas 0.3

% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.2
% d'aliments dans le top3 pour 1 repas 0.29
% d'aliments dans le top3 pour 1 repas 0.43
% d'aliments dans le top3 pour 1 repas 0.11
% d'aliments dans le top3 pour 1 repas 0.14
% d'aliments dans le top3 pour 1 repas 0.43
% d'aliments dans le top3 pour 1 repas 0.38
% d'aliments dans le top3 pour 1 repas 0.25
% d'aliments dans le top3 pour 1 repas 0.17
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.25
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.62
% d'aliments dans le top3 pour 1 repas 0.3
% d'aliments dans le top3 pour 1 repas 0.4
% d'aliments dans le top3 pour 1 repas 0.2
% d'aliments dans le top3 pour 1 repas 0.25
% d'aliments dans le top3 pour 1 repas 0.38
% d'aliments dans le top3 pour 1 repas 

% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.38
% d'aliments dans le top3 pour 1 repas 0.29
% d'aliments dans le top3 pour 1 repas 0.67
% d'aliments dans le top3 pour 1 repas 0.31
% d'aliments dans le top3 pour 1 repas 0.11
% d'aliments dans le top3 pour 1 repas 0.14
% d'aliments dans le top3 pour 1 repas 0.57
% d'aliments dans le top3 pour 1 repas 0.38
% d'aliments dans le top3 pour 1 repas 0.27
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.38
% d'aliments dans le top3 pour 1 repas 0.22
% d'aliments dans le top3 pour 1 repas 0.12
% d'aliments dans le top3 pour 1 repas 0.43
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.14
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas

('pourcentage global de bonne classification:', 0.35)

In [29]:
#evaluer performance sur le set de test: métrique de classification TOP 3 SANS LE EOS 


TEST_BATCHSIZE=1

def assess_classif(preprocessor, model, all_sentences):    
    count=0
    percent_ok=0
    realtest_loader = DataLoader(all_sentences[(len(train_data_sentences)+len(validation_data_sentences)):], TEST_BATCHSIZE, shuffle=False)
    model.eval()
    with torch.no_grad():
        for batch in realtest_loader:
            num_top3=0
            x = preprocessor.process_batch(batch)
#             print("x", x)
            x_old, y_old = x[:-1].to(device), x[1:].to(device)
            yhat_old, hidden = model(x_old)
            x, y = x_old[:-1], y_old[:-1]
            yhat = yhat_old[:-1, :, :]                  
#             print("x", x)
#             print("y", y)
#             print("y size", y.shape)
#             print("yhat size", yhat.shape)
#             print("yhat", yhat)
#             print("nombre d'aliments", yhat.shape[0])
            top_values, top_indices = torch.topk(yhat, k=3, dim=2)
#             print("top3 classes size", top_indices.shape)            
#             print("top3 classes", top_indices)
            for i in range(yhat.shape[0]):
                if y[i,0] in top_indices[i,0,:]:
                    num_top3 +=1
            print("% d'aliments dans le top3 pour 1 repas", round(num_top3/yhat.shape[0], 2))
            count += yhat.shape[0]
            percent_ok += num_top3
        return("pourcentage global de bonne classification DANS LE TOP3 SANS LE EOS:", round(percent_ok/count, 2))

assess_classif(preprocessor, rnn_model, all_sentences)

% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.29
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.4
% d'aliments dans le top3 pour 1 repas 0.2
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.2
% d'aliments dans le top3 pour 1 repas 0.2
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.17
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.25
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.25
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.57
% d'aliments dans le top3 pour 1 repas 0.17
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.57
% d'aliments dans le top3 pour 1 repas 0.25
% 

% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.44
% d'aliments dans le top3 pour 1 repas 0.25
% d'aliments dans le top3 pour 1 repas 0.22
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.2
% d'aliments dans le top3 pour 1 repas 0.43
% d'aliments dans le top3 pour 1 repas 0.11
% d'aliments dans le top3 pour 1 repas 0.22
% d'aliments dans le top3 pour 1 repas 0.2
% d'aliments dans le top3 pour 1 repas 0.29
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.27
% d'aliments dans le top3 pour 1 repas 0.2
% d'aliments dans le top3 pour 1 repas 0.17
% d'aliments dans le top3 pour 1 repas 0.45
% d'aliments dans le top3 pour 1 repas 0.

% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.38
% d'aliments dans le top3 pour 1 repas 0.17
% d'aliments dans le top3 pour 1 repas 0.22
% d'aliments dans le top3 pour 1 repas 0.3
% d'aliments dans le top3 pour 1 repas 0.12
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.17
% d'aliments dans le top3 pour 1 repas 0.43
% d'aliments dans le top3 pour 1 repas 0.22
% d'aliments dans le top3 pour 1 repas 0.25
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.2
% d'aliments dans le top3 pour 1 repas 0.3
% d'aliments dans le top3 pour 1 repas 0.2
% d'aliments dans le top3 pour 1 repas 0.25
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.17
% d'aliments dans le top3 pour 1 repas 0.2
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.21
% d'aliments dans le top3 pour 1 repas 0.33

% d'aliments dans le top3 pour 1 repas 0.23
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.6
% d'aliments dans le top3 pour 1 repas 0.29
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.44
% d'aliments dans le top3 pour 1 repas 0.17
% d'aliments dans le top3 pour 1 repas 0.09
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.29
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.1
% d'aliments dans le top3 pour 1 repas 0.25
% d'aliments dans le top3 pour 1 repas 0.2
% d'aliments dans le top3 pour 1 repas 0.27
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.4
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.33

% d'aliments dans le top3 pour 1 repas 0.08
% d'aliments dans le top3 pour 1 repas 0.4
% d'aliments dans le top3 pour 1 repas 0.1
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.67
% d'aliments dans le top3 pour 1 repas 1.0
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.25
% d'aliments dans le top3 pour 1 repas 0.12
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.4
% d'aliments dans le top3 pour 1 repas 0.08
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.8
% d'aliments dans le top3 pour 1 repas 0.6
% d'aliments dans le top3 pour 1 repas 0.33
% d'aliments dans le top3 pour 1 repas 0.17
% d'aliments dans le top3 pour 1 repas 0.0
% d'aliments dans le top3 pour 1 repas 0.14
% d'aliments dans le top3 pour 1 repas 0.5
% d'aliments dans le top3 pour 1 repas 0.8
% 

('pourcentage global de bonne classification DANS LE TOP3 SANS LE EOS:', 0.25)

In [30]:
#evaluer performance sur le set de test: métrique de classification / PERFORMANCE AU DELA DU 3E ALIMENT PREDIT 

TEST_BATCHSIZE=1

def assess_classif_byw(preprocessor, model, all_sentences):    
    count_w1 =0
    count_w2 =0
    count_w3 =0
    count_w4 =0
    nb_ok_w1 =0
    nb_ok_w2 =0
    nb_ok_w3 =0
    nb_ok_w4 =0 
    realtest_loader = DataLoader(all_sentences[(len(train_data_sentences)+len(validation_data_sentences)):], TEST_BATCHSIZE, shuffle=False)
    model.eval()
    with torch.no_grad():
        for batch in realtest_loader:
            x = preprocessor.process_batch(batch)
#             print("x", x)
            x, y = x[:-1].to(device), x[1:].to(device)
            yhat, hidden = model(x)
#             print("x", x)
#             print("y", y)
#             print("yhat size", yhat.shape)
#             print("yhat", yhat)
#             print("nombre d'aliments", yhat.shape[0])
            n=yhat.shape[0]
            if n>3:
                print("n=", n)
                yhat_w1=yhat[0:1]
                yhat_w2=yhat[1:2]
                yhat_w3=yhat[2:3]
                yhat_w4=yhat[3:]
                y_w1=y[0:1]
                y_w2=y[1:2]
                y_w3=y[2:3]
                y_w4=y[3:]
                print(yhat_w1.shape)             
                print(yhat_w2.shape) 
                print(yhat_w3.shape) 
                print(yhat_w4.shape) 
                print(y_w1.shape)             
                print(y_w2.shape) 
                print(y_w3.shape) 
                print(y_w4.shape)                 
                pred_class_w1 = torch.argmax(yhat_w1, dim=2)
                pred_class_w2 = torch.argmax(yhat_w2, dim=2)
                pred_class_w3 = torch.argmax(yhat_w3, dim=2)
                pred_class_w4 = torch.argmax(yhat_w4, dim=2)
                num_ewual_w1 = (pred_class_w1 == y_w1).sum().item()
                num_ewual_w2 = (pred_class_w2 == y_w2).sum().item()    
                num_ewual_w3 = (pred_class_w3 == y_w3).sum().item()
                num_ewual_w4 = (pred_class_w4 == y_w4).sum().item()      
                count_w1 += yhat_w1.shape[0]
                count_w2 += yhat_w2.shape[0]
                count_w3 += yhat_w3.shape[0]
                count_w4 += yhat_w4.shape[0]
                nb_ok_w1 += num_ewual_w1
                nb_ok_w2 += num_ewual_w2    
                nb_ok_w3 += num_ewual_w3
                nb_ok_w4 += num_ewual_w4
                
        print("pourcentage global de bonne classification w1:", round(nb_ok_w1/count_w1, 2))
        print("pourcentage global de bonne classification w2:", round(nb_ok_w2/count_w2, 2))
        print("pourcentage global de bonne classification w3:", round(nb_ok_w3/count_w3, 2))
        print("pourcentage global de bonne classification w4:", round(nb_ok_w4/count_w4, 2))
        print("counts", count_w1, count_w2, count_w3, count_w4)
        print("nok", nb_ok_w1, nb_ok_w2, nb_ok_w3, nb_ok_w4)
        
        
assess_classif_byw(preprocessor, rnn_model, all_sentences)


n= 7
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([4, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([4, 1])
n= 8
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([5, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([5, 1])
n= 7
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([4, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([4, 1])
n= 9
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([6, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([6, 1])
n= 6
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([3, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([3, 1])
n= 6
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([3, 1, 339])
torch.Size([1,

n= 9
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([6, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([6, 1])
n= 7
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([4, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([4, 1])
n= 10
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([7, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([7, 1])
n= 6
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([3, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([3, 1])
n= 6
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([3, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([3, 1])
n= 8
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([5, 1, 339])
torch.Size([1

n= 9
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([6, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([6, 1])
n= 7
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([4, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([4, 1])
n= 8
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([5, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([5, 1])
n= 7
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([4, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([4, 1])
n= 10
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([7, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([7, 1])
n= 7
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([4, 1, 339])
torch.Size([1

n= 7
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([4, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([4, 1])
n= 4
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
n= 8
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([5, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([5, 1])
n= 5
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([2, 1])
n= 5
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([2, 1])
n= 6
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([3, 1, 339])
torch.Size([1,

In [31]:
#evaluer performance sur le set de test: métrique de classification / 
# PERFORMANCE AU DELA DU 3E ALIMENT PREDIT SANS LE EOS

TEST_BATCHSIZE=1

def assess_classif_byw(preprocessor, model, all_sentences):    
    count_w1 =0
    count_w2 =0
    count_w3 =0
    count_w4 =0
    nb_ok_w1 =0
    nb_ok_w2 =0
    nb_ok_w3 =0
    nb_ok_w4 =0 
    realtest_loader = DataLoader(all_sentences[(len(train_data_sentences)+len(validation_data_sentences)):], TEST_BATCHSIZE, shuffle=False)
    model.eval()
    with torch.no_grad():
        for batch in realtest_loader:
            x = preprocessor.process_batch(batch)
#             print("x", x)
            x_old, y_old = x[:-1].to(device), x[1:].to(device)
            yhat_old, hidden = model(x_old)
            x, y = x_old[:-1], y_old[:-1]
            yhat = yhat_old[:-1, :, :]       
#             print("x", x)
#             print("y", y)
#             print("yhat size", yhat.shape)
#             print("yhat", yhat)
#             print("nombre d'aliments", yhat.shape[0])
            n=yhat.shape[0]
            if n>3:
                print("n=", n)
                yhat_w1=yhat[0:1]
                yhat_w2=yhat[1:2]
                yhat_w3=yhat[2:3]
                yhat_w4=yhat[3:]
                y_w1=y[0:1]
                y_w2=y[1:2]
                y_w3=y[2:3]
                y_w4=y[3:]
                print(yhat_w1.shape)             
                print(yhat_w2.shape) 
                print(yhat_w3.shape) 
                print(yhat_w4.shape) 
                print(y_w1.shape)             
                print(y_w2.shape) 
                print(y_w3.shape) 
                print(y_w4.shape)                 
                pred_class_w1 = torch.argmax(yhat_w1, dim=2)
                pred_class_w2 = torch.argmax(yhat_w2, dim=2)
                pred_class_w3 = torch.argmax(yhat_w3, dim=2)
                pred_class_w4 = torch.argmax(yhat_w4, dim=2)
                num_ewual_w1 = (pred_class_w1 == y_w1).sum().item()
                num_ewual_w2 = (pred_class_w2 == y_w2).sum().item()    
                num_ewual_w3 = (pred_class_w3 == y_w3).sum().item()
                num_ewual_w4 = (pred_class_w4 == y_w4).sum().item()      
                count_w1 += yhat_w1.shape[0]
                count_w2 += yhat_w2.shape[0]
                count_w3 += yhat_w3.shape[0]
                count_w4 += yhat_w4.shape[0]
                nb_ok_w1 += num_ewual_w1
                nb_ok_w2 += num_ewual_w2    
                nb_ok_w3 += num_ewual_w3
                nb_ok_w4 += num_ewual_w4
                
        print("pourcentage global de bonne classification w1 SANS EOS:", round(nb_ok_w1/count_w1, 2))
        print("pourcentage global de bonne classification w2 SANS EOS:", round(nb_ok_w2/count_w2, 2))
        print("pourcentage global de bonne classification w3 SANS EOS:", round(nb_ok_w3/count_w3, 2))
        print("pourcentage global de bonne classification w4 SANS EOS:", round(nb_ok_w4/count_w4, 2))
        print("counts", count_w1, count_w2, count_w3, count_w4)
        print("nok", nb_ok_w1, nb_ok_w2, nb_ok_w3, nb_ok_w4)
        
        
assess_classif_byw(preprocessor, rnn_model, all_sentences)


n= 6
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([3, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([3, 1])
n= 7
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([4, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([4, 1])
n= 6
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([3, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([3, 1])
n= 8
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([5, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([5, 1])
n= 5
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([2, 1])
n= 5
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1,

n= 7
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([4, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([4, 1])
n= 7
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([4, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([4, 1])
n= 4
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
n= 10
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([7, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([7, 1])
n= 9
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([6, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([6, 1])
n= 6
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([3, 1, 339])
torch.Size([1

n= 8
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([5, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([5, 1])
n= 6
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([3, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([3, 1])
n= 7
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([4, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([4, 1])
n= 6
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([3, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([3, 1])
n= 9
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([6, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([6, 1])
n= 6
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([3, 1, 339])
torch.Size([1,

n= 7
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([4, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([4, 1])
n= 7
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([4, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([4, 1])
n= 6
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([3, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([3, 1])
n= 4
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
n= 13
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([10, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([10, 1])
n= 8
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([5, 1, 339])
torch.Size(

In [32]:
#evaluer performance sur le set test: métrique classif 
# PERFORMANCE DANS LE TOP 3 AU DELA DU 3E ALIMENT PREDIT SANS LE EOS

TEST_BATCHSIZE=1

def assess_classif_byw(preprocessor, model, all_sentences):    
    count_w1 =0
    count_w2 =0
    count_w3 =0
    count_w4 =0
    nb_ok_w1 =0
    nb_ok_w2 =0
    nb_ok_w3 =0
    nb_ok_w4 =0 
    realtest_loader = DataLoader(all_sentences[(len(train_data_sentences)+len(validation_data_sentences)):], TEST_BATCHSIZE, shuffle=False)
    model.eval()
    with torch.no_grad():
        for batch in realtest_loader:
            x = preprocessor.process_batch(batch)
#             print("x", x)
            x_old, y_old = x[:-1].to(device), x[1:].to(device)
            yhat_old, hidden = model(x_old)
            x, y = x_old[:-1], y_old[:-1]
            yhat = yhat_old[:-1, :, :]       
#             print("x", x)
#             print("y", y)
#             print("yhat size", yhat.shape)
#             print("yhat", yhat)
#             print("nombre d'aliments", yhat.shape[0])
            n=yhat.shape[0]
            if n>3:
                num_ewual_w4=0
                print("n=", n)
                yhat_w1=yhat[0:1]
                yhat_w2=yhat[1:2]
                yhat_w3=yhat[2:3]
                yhat_w4=yhat[3:]
                y_w1=y[0:1]
                y_w2=y[1:2]
                y_w3=y[2:3]
                y_w4=y[3:]
                print(yhat_w1.shape)             
                print(yhat_w2.shape) 
                print(yhat_w3.shape) 
                print(yhat_w4.shape) 
                print(y_w1.shape)             
                print(y_w2.shape) 
                print(y_w3.shape) 
                print(y_w4.shape) 
                top_valw1, top_indw1=torch.topk(yhat_w1, k=3, dim=2)
                top_valw2, top_indw2=torch.topk(yhat_w2, k=3, dim=2)
                top_valw3, top_indw3=torch.topk(yhat_w3, k=3, dim=2)
                top_valw4, top_indw4=torch.topk(yhat_w4, k=3, dim=2)
                print("y_w1", y_w1[0,0])
                print("y_w4", y_w4[0,0])
                print("top_indw1", top_indw1[0,0,:])
                print("top_indw4", top_indw4[0,0,:])                
                num_ewual_w1 = (top_indw1[0,0,:] == y_w1[0,0]).sum().item()
                num_ewual_w2 = (top_indw2[0,0,:] == y_w2[0,0]).sum().item()    
                num_ewual_w3 = (top_indw3[0,0,:] == y_w3[0,0]).sum().item()
                for i in range(yhat_w4.shape[0]):
                    if y_w4[i,0] in top_indw4[i,0,:]:
                         num_ewual_w4 +=1
                count_w1 += yhat_w1.shape[0]
                count_w2 += yhat_w2.shape[0]
                count_w3 += yhat_w3.shape[0]
                count_w4 += yhat_w4.shape[0]
                nb_ok_w1 += num_ewual_w1
                nb_ok_w2 += num_ewual_w2    
                nb_ok_w3 += num_ewual_w3
                nb_ok_w4 += num_ewual_w4
                
        print("pourcentage global de bonne classification w1 dans le top 3 SANS EOS:", round(nb_ok_w1/count_w1, 2))
        print("pourcentage global de bonne classification w2 dans le top 3 SANS EOS:", round(nb_ok_w2/count_w2, 2))
        print("pourcentage global de bonne classification w3 dans le top 3 SANS EOS:", round(nb_ok_w3/count_w3, 2))
        print("pourcentage global de bonne classification w4 dans le top 3 SANS EOS:", round(nb_ok_w4/count_w4, 2))
        print("counts", count_w1, count_w2, count_w3, count_w4)
        print("nok", nb_ok_w1, nb_ok_w2, nb_ok_w3, nb_ok_w4)
        
        
assess_classif_byw(preprocessor, rnn_model, all_sentences)


n= 6
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([3, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([3, 1])
y_w1 tensor(145)
y_w4 tensor(50)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([76, 34,  5])
n= 7
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([4, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([4, 1])
y_w1 tensor(7)
y_w4 tensor(101)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([6, 5, 8])
n= 6
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([3, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([3, 1])
y_w1 tensor(44)
y_w4 tensor(237)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([ 5, 13,  6])
n= 8
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([5, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([5, 1])
y_w1 tensor

top_indw1 tensor([17,  7,  9])
top_indw4 tensor([ 5, 49,  4])
n= 7
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([4, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([4, 1])
y_w1 tensor(14)
y_w4 tensor(4)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([ 4,  5, 11])
n= 4
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
y_w1 tensor(232)
y_w4 tensor(31)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([ 3,  4, 11])
n= 10
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([7, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([7, 1])
y_w1 tensor(74)
y_w4 tensor(97)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([ 85, 221,  45])
n= 6
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([3, 1, 339])
torch.Size([1, 1])
t

n= 4
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
y_w1 tensor(53)
y_w4 tensor(6)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([3, 5, 6])
n= 6
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([3, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([3, 1])
y_w1 tensor(61)
y_w4 tensor(65)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([5, 6, 3])
n= 9
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([6, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([6, 1])
y_w1 tensor(256)
y_w4 tensor(47)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([ 4,  5, 22])
n= 4
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
y_w1 tensor(46)


y_w1 tensor(48)
y_w4 tensor(5)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([5, 6, 3])
n= 4
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
y_w1 tensor(4)
y_w4 tensor(16)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([ 6,  5, 11])
n= 6
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([3, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([3, 1])
y_w1 tensor(5)
y_w4 tensor(4)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([ 3,  4, 19])
n= 5
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([2, 1])
y_w1 tensor(13)
y_w4 tensor(38)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([4, 5, 3])
n= 13
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([10, 1, 339])


n= 7
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([4, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([4, 1])
y_w1 tensor(17)
y_w4 tensor(218)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([94, 81, 76])
n= 10
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([7, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([7, 1])
y_w1 tensor(7)
y_w4 tensor(13)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([76,  5, 19])
n= 4
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
y_w1 tensor(35)
y_w4 tensor(105)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([5, 6, 4])
n= 4
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
y_w1 tensor

n= 5
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([2, 1])
y_w1 tensor(86)
y_w4 tensor(5)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([72,  5,  4])
n= 7
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([4, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([4, 1])
y_w1 tensor(37)
y_w4 tensor(5)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([4, 5, 8])
n= 9
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([6, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([6, 1])
y_w1 tensor(17)
y_w4 tensor(19)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([ 4,  3, 31])
n= 4
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
y_w1 tensor(220

n= 12
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([9, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([9, 1])
y_w1 tensor(35)
y_w4 tensor(186)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([5, 4, 6])
n= 4
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
y_w1 tensor(133)
y_w4 tensor(38)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([4, 5, 6])
n= 8
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([5, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([5, 1])
y_w1 tensor(42)
y_w4 tensor(141)
top_indw1 tensor([17,  7,  9])
top_indw4 tensor([13, 61,  6])
n= 4
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
y_w1 tensor(

In [33]:
#evaluer performance sur le set de test: métrique de classification / PERFORMANCE PAR QUARTILE

TEST_BATCHSIZE=1

def assess_classif_byQ(preprocessor, model, all_sentences):    
    count_q1 =0
    count_q2 =0
    count_q3 =0
    count_q4 =0
    nb_ok_q1 =0
    nb_ok_q2 =0
    nb_ok_q3 =0
    nb_ok_q4 =0 
    realtest_loader = DataLoader(all_sentences[(len(train_data_sentences)+len(validation_data_sentences)):], TEST_BATCHSIZE, shuffle=False)
    model.eval()
    with torch.no_grad():
        for batch in realtest_loader:
            x = preprocessor.process_batch(batch)
#             print("x", x)
            x, y = x[:-1].to(device), x[1:].to(device)
            yhat, hidden = model(x)
#             print("x", x)
#             print("y", y)
#             print("yhat size", yhat.shape)
#             print("yhat", yhat)
#             print("nombre d'aliments", yhat.shape[0])
            n=yhat.shape[0]
            if n>3:
                print("n=", n)
                yhat_q1=yhat[0:round(n/4)]
                yhat_q2=yhat[round(n/4):round(n/2)]
                yhat_q3=yhat[round(n/2):round(3*n/4)]
                yhat_q4=yhat[round(3*n/4):]
                y_q1=y[0:round(n/4)]
                y_q2=y[round(n/4):round(n/2)]
                y_q3=y[round(n/2):round(3*n/4)]
                y_q4=y[round(3*n/4):]        
                print(yhat_q1.shape)             
                print(yhat_q2.shape) 
                print(yhat_q3.shape) 
                print(yhat_q4.shape) 
                print(y_q1.shape)             
                print(y_q2.shape) 
                print(y_q3.shape) 
                print(y_q4.shape)                 
                pred_class_q1 = torch.argmax(yhat_q1, dim=2)
                pred_class_q2 = torch.argmax(yhat_q2, dim=2)
                pred_class_q3 = torch.argmax(yhat_q3, dim=2)
                pred_class_q4 = torch.argmax(yhat_q4, dim=2)
                num_equal_q1 = (pred_class_q1 == y_q1).sum().item()
                num_equal_q2 = (pred_class_q2 == y_q2).sum().item()    
                num_equal_q3 = (pred_class_q3 == y_q3).sum().item()
                num_equal_q4 = (pred_class_q4 == y_q4).sum().item()      
                count_q1 += yhat_q1.shape[0]
                count_q2 += yhat_q2.shape[0]
                count_q3 += yhat_q3.shape[0]
                count_q4 += yhat_q4.shape[0]
                nb_ok_q1 += num_equal_q1
                nb_ok_q2 += num_equal_q2    
                nb_ok_q3 += num_equal_q3
                nb_ok_q4 += num_equal_q4
                
        print("pourcentage global de bonne classification q1:", round(nb_ok_q1/count_q1, 2))
        print("pourcentage global de bonne classification q2:", round(nb_ok_q2/count_q2, 2))
        print("pourcentage global de bonne classification q3:", round(nb_ok_q3/count_q3, 2))
        print("pourcentage global de bonne classification q4:", round(nb_ok_q4/count_q4, 2))
        print("counts", count_q1, count_q2, count_q3, count_q4)
        print("nok", nb_ok_q1, nb_ok_q2, nb_ok_q3, nb_ok_q4)
        
assess_classif_byQ(preprocessor, rnn_model, all_sentences)


n= 7
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size([2, 1])
n= 8
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([2, 1])
n= 7
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size([2, 1])
n= 9
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([3, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([3, 1])
torch.Size([2, 1])
n= 6
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([2, 1])
n= 6
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2,

n= 9
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([3, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([3, 1])
torch.Size([2, 1])
n= 7
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size([2, 1])
n= 10
torch.Size([2, 1, 339])
torch.Size([3, 1, 339])
torch.Size([3, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([3, 1])
torch.Size([3, 1])
torch.Size([2, 1])
n= 6
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([2, 1])
n= 6
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([2, 1])
n= 8
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2

n= 7
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size([2, 1])
n= 10
torch.Size([2, 1, 339])
torch.Size([3, 1, 339])
torch.Size([3, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([3, 1])
torch.Size([3, 1])
torch.Size([2, 1])
n= 11
torch.Size([3, 1, 339])
torch.Size([3, 1, 339])
torch.Size([2, 1, 339])
torch.Size([3, 1, 339])
torch.Size([3, 1])
torch.Size([3, 1])
torch.Size([2, 1])
torch.Size([3, 1])
n= 9
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([3, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([3, 1])
torch.Size([2, 1])
n= 9
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([3, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([3, 1])
torch.Size([2, 1])
n= 7
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([

n= 8
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([2, 1])
n= 7
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size([2, 1])
n= 5
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([2, 1])
torch.Size([1, 1])
n= 7
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size([2, 1])
n= 9
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([3, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([3, 1])
torch.Size([2, 1])
n= 5
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1,

n= 8
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([2, 1])
n= 7
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size([2, 1])
n= 7
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size([2, 1])
n= 10
torch.Size([2, 1, 339])
torch.Size([3, 1, 339])
torch.Size([3, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([3, 1])
torch.Size([3, 1])
torch.Size([2, 1])
n= 13
torch.Size([3, 1, 339])
torch.Size([3, 1, 339])
torch.Size([4, 1, 339])
torch.Size([3, 1, 339])
torch.Size([3, 1])
torch.Size([3, 1])
torch.Size([4, 1])
torch.Size([3, 1])
n= 4
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([

In [24]:
#evaluer performance sur le set de test: métrique de classification 
# PERFORMANCE PAR QUARTILE SANS LE EOS

TEST_BATCHSIZE=1

def assess_classif_byQ(preprocessor, model, all_sentences):    
    count_q1 =0
    count_q2 =0
    count_q3 =0
    count_q4 =0
    nb_ok_q1 =0
    nb_ok_q2 =0
    nb_ok_q3 =0
    nb_ok_q4 =0 
    realtest_loader = DataLoader(all_sentences[(len(train_data_sentences)+len(validation_data_sentences)):], TEST_BATCHSIZE, shuffle=False)
    model.eval()
    with torch.no_grad():
        for batch in realtest_loader:
            x = preprocessor.process_batch(batch)
#             print("x", x)
            x_old, y_old = x[:-1].to(device), x[1:].to(device)
            yhat_old, hidden = model(x_old)
            x, y = x_old[:-1], y_old[:-1]
            yhat = yhat_old[:-1, :, :]      
            print("x", x)
            print("y", y)
            print("yhat size", yhat.shape)
#             print("yhat", yhat)
            print("nombre d'aliments", yhat.shape[0])
            n=yhat.shape[0]
            if n>3:
                print("n=", n)
                yhat_q1=yhat[0:round(n/4)]
                yhat_q2=yhat[round(n/4):round(n/2)]
                yhat_q3=yhat[round(n/2):round(3*n/4)]
                yhat_q4=yhat[round(3*n/4):]
                y_q1=y[0:round(n/4)]
                y_q2=y[round(n/4):round(n/2)]
                y_q3=y[round(n/2):round(3*n/4)]
                y_q4=y[round(3*n/4):]        
                print(yhat_q1.shape)             
                print(yhat_q2.shape) 
                print(yhat_q3.shape) 
                print(yhat_q4.shape) 
                print(y_q1.shape)             
                print(y_q2.shape) 
                print(y_q3.shape) 
                print(y_q4.shape)                 
                pred_class_q1 = torch.argmax(yhat_q1, dim=2)
                pred_class_q2 = torch.argmax(yhat_q2, dim=2)
                pred_class_q3 = torch.argmax(yhat_q3, dim=2)
                pred_class_q4 = torch.argmax(yhat_q4, dim=2)
                print("pred_class_q1", pred_class_q1)
                print("pred_class_q2", pred_class_q2)
                print("pred_class_q3", pred_class_q3)
                print("pred_class_q4", pred_class_q4)
                num_equal_q1 = (pred_class_q1 == y_q1).sum().item()
                num_equal_q2 = (pred_class_q2 == y_q2).sum().item()    
                num_equal_q3 = (pred_class_q3 == y_q3).sum().item()
                num_equal_q4 = (pred_class_q4 == y_q4).sum().item()      
                count_q1 += yhat_q1.shape[0]
                count_q2 += yhat_q2.shape[0]
                count_q3 += yhat_q3.shape[0]
                count_q4 += yhat_q4.shape[0]
                nb_ok_q1 += num_equal_q1
                nb_ok_q2 += num_equal_q2    
                nb_ok_q3 += num_equal_q3
                nb_ok_q4 += num_equal_q4
                
        print("pourcentage global de bonne classification q1 sans EOS:", round(nb_ok_q1/count_q1, 2))
        print("pourcentage global de bonne classification q2 sans EOS:", round(nb_ok_q2/count_q2, 2))
        print("pourcentage global de bonne classification q3 sans EOS:", round(nb_ok_q3/count_q3, 2))
        print("pourcentage global de bonne classification q4 sans EOS:", round(nb_ok_q4/count_q4, 2))
        print("counts", count_q1, count_q2, count_q3, count_q4)
        print("nok", nb_ok_q1, nb_ok_q2, nb_ok_q3, nb_ok_q4)
        
assess_classif_byQ(preprocessor, rnn_model, all_sentences)


x tensor([[  2],
        [145],
        [ 36],
        [ 94],
        [ 50],
        [198]])
y tensor([[145],
        [ 36],
        [ 94],
        [ 50],
        [198],
        [ 58]])
yhat size torch.Size([6, 1, 339])
nombre d'aliments 6
n= 6
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([2, 1])
pred_class_q1 tensor([[17],
        [19]])
pred_class_q2 tensor([[35]])
pred_class_q3 tensor([[241]])
pred_class_q4 tensor([[5],
        [5]])
x tensor([[  2],
        [  7],
        [223],
        [ 40],
        [101],
        [206],
        [  5]])
y tensor([[  7],
        [223],
        [ 40],
        [101],
        [206],
        [  5],
        [ 66]])
yhat size torch.Size([7, 1, 339])
nombre d'aliments 7
n= 7
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size([

x tensor([[  2],
        [293],
        [ 82],
        [288],
        [141],
        [132],
        [115]])
y tensor([[293],
        [ 82],
        [288],
        [141],
        [132],
        [115],
        [ 39]])
yhat size torch.Size([7, 1, 339])
nombre d'aliments 7
n= 7
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size([2, 1])
pred_class_q1 tensor([[17],
        [18]])
pred_class_q2 tensor([[18],
        [18]])
pred_class_q3 tensor([[5]])
pred_class_q4 tensor([[4],
        [3]])
x tensor([[  2],
        [169],
        [  7],
        [ 19],
        [ 22],
        [ 31]])
y tensor([[169],
        [  7],
        [ 19],
        [ 22],
        [ 31],
        [ 56]])
yhat size torch.Size([6, 1, 339])
nombre d'aliments 6
n= 6
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size([1, 1])


x tensor([[  2],
        [ 14],
        [ 62],
        [ 88],
        [252],
        [ 22],
        [ 43],
        [  4]])
y tensor([[ 14],
        [ 62],
        [ 88],
        [252],
        [ 22],
        [ 43],
        [  4],
        [  6]])
yhat size torch.Size([8, 1, 339])
nombre d'aliments 8
n= 8
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([2, 1])
pred_class_q1 tensor([[17],
        [18]])
pred_class_q2 tensor([[27],
        [ 4]])
pred_class_q3 tensor([[4],
        [4]])
pred_class_q4 tensor([[5],
        [5]])
x tensor([[ 2],
        [37],
        [52],
        [22],
        [32],
        [ 6]])
y tensor([[ 37],
        [ 52],
        [ 22],
        [ 32],
        [  6],
        [105]])
yhat size torch.Size([6, 1, 339])
nombre d'aliments 6
n= 6
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
t

x tensor([[  2],
        [256],
        [ 37],
        [ 19],
        [ 47],
        [ 34],
        [  4],
        [  5],
        [ 12]])
y tensor([[256],
        [ 37],
        [ 19],
        [ 47],
        [ 34],
        [  4],
        [  5],
        [ 12],
        [ 12]])
yhat size torch.Size([9, 1, 339])
nombre d'aliments 9
n= 9
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([3, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([3, 1])
torch.Size([2, 1])
pred_class_q1 tensor([[17],
        [ 4]])
pred_class_q2 tensor([[7],
        [4]])
pred_class_q3 tensor([[5],
        [4],
        [5]])
pred_class_q4 tensor([[3],
        [3]])
x tensor([[  2],
        [ 46],
        [192],
        [ 38]])
y tensor([[ 46],
        [192],
        [ 38],
        [ 39]])
yhat size torch.Size([4, 1, 339])
nombre d'aliments 4
n= 4
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1])
torch.Size([1,

x tensor([[  2],
        [ 88],
        [138],
        [ 83],
        [ 54],
        [ 33],
        [ 42],
        [  5]])
y tensor([[ 88],
        [138],
        [ 83],
        [ 54],
        [ 33],
        [ 42],
        [  5],
        [ 39]])
yhat size torch.Size([8, 1, 339])
nombre d'aliments 8
n= 8
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([2, 1])
pred_class_q1 tensor([[17],
        [13]])
pred_class_q2 tensor([[4],
        [5]])
pred_class_q3 tensor([[5],
        [5]])
pred_class_q4 tensor([[5],
        [3]])
x tensor([[  2],
        [ 41],
        [184],
        [104],
        [ 43],
        [ 23]])
y tensor([[ 41],
        [184],
        [104],
        [ 43],
        [ 23],
        [146]])
yhat size torch.Size([6, 1, 339])
nombre d'aliments 6
n= 6
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1

...

x tensor([[ 2],
        [17]])
y tensor([[17],
        [ 6]])
yhat size torch.Size([2, 1, 339])
nombre d'aliments 2
x tensor([[  2],
        [193],
        [ 48],
        [211],
        [  8]])
y tensor([[193],
        [ 48],
        [211],
        [  8],
        [132]])
yhat size torch.Size([5, 1, 339])
nombre d'aliments 5
n= 5
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([2, 1])
torch.Size([1, 1])
pred_class_q1 tensor([[17]])
pred_class_q2 tensor([[57]])
pred_class_q3 tensor([[5],
        [5]])
pred_class_q4 tensor([[16]])
x tensor([[  2],
        [ 24],
        [  7],
        [  5],
        [ 69],
        [ 72],
        [177],
        [  4]])
y tensor([[ 24],
        [  7],
        [  5],
        [ 69],
        [ 72],
        [177],
        [  4],
        [ 10]])
yhat size torch.Size([8, 1, 339])
nombre d'aliments 8
n= 8
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1

x tensor([[ 2],
        [17],
        [25]])
y tensor([[ 17],
        [ 25],
        [224]])
yhat size torch.Size([3, 1, 339])
nombre d'aliments 3
x tensor([[ 2],
        [ 5],
        [50],
        [29]])
y tensor([[ 5],
        [50],
        [29],
        [ 8]])
yhat size torch.Size([4, 1, 339])
nombre d'aliments 4
n= 4
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
pred_class_q1 tensor([[17]])
pred_class_q2 tensor([[4]])
pred_class_q3 tensor([[9]])
pred_class_q4 tensor([[16]])
x tensor([[  2],
        [157],
        [ 87],
        [ 47],
        [ 19],
        [ 97]])
y tensor([[157],
        [ 87],
        [ 47],
        [ 19],
        [ 97],
        [ 12]])
yhat size torch.Size([6, 1, 339])
nombre d'aliments 6
n= 6
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size(

x tensor([[  2],
        [166],
        [  5],
        [  4],
        [  9],
        [ 18],
        [ 48],
        [251],
        [295],
        [217],
        [281]])
y tensor([[166],
        [  5],
        [  4],
        [  9],
        [ 18],
        [ 48],
        [251],
        [295],
        [217],
        [281],
        [115]])
yhat size torch.Size([11, 1, 339])
nombre d'aliments 11
n= 11
torch.Size([3, 1, 339])
torch.Size([3, 1, 339])
torch.Size([2, 1, 339])
torch.Size([3, 1, 339])
torch.Size([3, 1])
torch.Size([3, 1])
torch.Size([2, 1])
torch.Size([3, 1])
pred_class_q1 tensor([[17],
        [ 4],
        [ 3]])
pred_class_q2 tensor([[ 3],
        [18],
        [ 3]])
pred_class_q3 tensor([[ 3],
        [62]])
pred_class_q4 tensor([[11],
        [ 3],
        [ 3]])
x tensor([[  2],
        [300],
        [242],
        [245],
        [190],
        [  5],
        [ 20]])
y tensor([[300],
        [242],
        [245],
        [190],
        [  5],
        [ 20],
        [ 10]])


x tensor([[  2],
        [184],
        [ 88],
        [163],
        [  8],
        [178],
        [  6]])
y tensor([[184],
        [ 88],
        [163],
        [  8],
        [178],
        [  6],
        [151]])
yhat size torch.Size([7, 1, 339])
nombre d'aliments 7
n= 7
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size([2, 1])
pred_class_q1 tensor([[17],
        [ 7]])
pred_class_q2 tensor([[4],
        [5]])
pred_class_q3 tensor([[16]])
pred_class_q4 tensor([[4],
        [3]])
x tensor([[  2],
        [204],
        [204]])
y tensor([[204],
        [204],
        [  5]])
yhat size torch.Size([3, 1, 339])
nombre d'aliments 3
x tensor([[  2],
        [ 17],
        [  9],
        [ 20],
        [ 27],
        [266],
        [  4],
        [ 28],
        [ 45]])
y tensor([[ 17],
        [  9],
        [ 20],
        [ 27],
        [266],
        [  4],
        [ 28],
    

y tensor([[242],
        [ 44],
        [  4],
        [ 65],
        [129],
        [ 18],
        [ 19]])
yhat size torch.Size([7, 1, 339])
nombre d'aliments 7
n= 7
torch.Size([2, 1, 339])
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size([2, 1])
pred_class_q1 tensor([[17],
        [14]])
pred_class_q2 tensor([[4],
        [5]])
pred_class_q3 tensor([[3]])
pred_class_q4 tensor([[16],
        [ 3]])
x tensor([[  2],
        [184],
        [ 57],
        [ 50],
        [  6],
        [ 11]])
y tensor([[184],
        [ 57],
        [ 50],
        [  6],
        [ 11],
        [ 71]])
yhat size torch.Size([6, 1, 339])
nombre d'aliments 6
n= 6
torch.Size([2, 1, 339])
torch.Size([1, 1, 339])
torch.Size([1, 1, 339])
torch.Size([2, 1, 339])
torch.Size([2, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([2, 1])
pred_class_q1 tensor([[17],
        [ 7]])
pred_class_q2 tensor([[9]])
pred_class_q3 tens

In [11]:
#ADAPTATION EXERSYS 
import os

# def save_model(preprocessor, model,fichier): # pas de sauvegarde de l'optimiseur ici
#       """ sauvegarde du modèle dans fichier """
#       state = {'model_state': model.state_dict(), 'preprocessor': preprocessor}
#       torch.save(state,fichier) # pas besoin de passer par pickle
 
# def load_model_RNN(fichier):
#       """ Si le fichier existe, on charge le modèle  """
#       if os.path.isfile(fichier):
#           state = torch.load(fichier)
#           preprocessor = state['preprocessor']
#           embeddings = preprocessor.embeddings(10)
#           model = RNN(embeddings, 100)
#           model.load_state_dict(state['model_state'])
#           return preprocessor, model

In [12]:
# #ADAPTATION EXERSYS 
# sauvegarde
# save_model(preprocessor, rnn_model, "model/rnn-gen-exersys_22epochs_50emb100hid")

# chargement ATTENTION à ne pas écraser un modèle appris
# preprocessor2,rnn_model2  = load_model_RNN("model/rnn-gen-exersys_500epochs_10emb100hid")

In [34]:
#ADAPTATION EXERSYS 
def generate(preprocessor: Preprocessor, model, start="it is", maxlength=26):
    """Génère une suite de tokens en utilisant la distribution de probabilité du modèle"""

    assert start, "le début de la phrase doit être indiqué"

    with torch.no_grad():
        x = preprocessor.process_batch([start]).to(device)
        generated = []
        y_t, s_t = model(x)
        # for borné sur maxlength
        #   tirage d'un mot selon les y
        #   si le mot n'est pas eos
        #       reappeler le modèle (il faut un .unsqueeze(0) sur le mot tiré)
        
        # <CORRECTION>
        for _ in range(maxlength):
            w_t = torch.distributions.categorical.Categorical(logits=y_t[-1]).sample() # tirage selon la distribution
            # w_t = torch.argmax(y_t[-1]).unsqueeze(0)
            # print(w_t, preprocessor.decode([w_t]))
            generated.append(w_t.item())
            if generated[-1] == preprocessor.eos_index:
                break

            x = w_t.unsqueeze(0)
            y_t, s_t = model(x,  s_t)
        # </CORRECTION>
        # print(generated) # liste d'indices
        return start + " " + preprocessor.decode(generated)

result=generate(preprocessor, rnn_model, start="25127")
print(result)



25127 7001 18066 18004 31017


In [35]:
# Chaîne de caractères contenant les nombres
chaine_de_caracteres = result

# Diviser la chaîne en sous-chaînes en utilisant l'espace comme séparateur
sous_chaines = chaine_de_caracteres.split()

# Convertir chaque sous-chaîne en entier
liste_d_entiers = [int(s) for s in sous_chaines]

# Afficher la liste d'entiers résultante
print(liste_d_entiers)

for e in liste_d_entiers:
    if e in dic_alim:
        print(dic_alim[e])
    else:
        print("pas dans le vocabulaire")

[25127, 7001, 18066, 18004, 31017]
couscous royal à la viande
pain baguette
eau du robinet
café noir prêt à boire non sucré
sucre roux


In [36]:
# Chaîne de caractères contenant les nombres
chaine_de_caracteres = result

# Diviser la chaîne en sous-chaînes en utilisant l'espace comme séparateur
liste_de_chaines = chaine_de_caracteres.split()

# Liste des repas qui contiennent le mot d'entrée et au moins un des autres mots générés
rep_gen=liste_de_chaines
for element in REP_vraisalimfreqliste:
    repas=[]
    if len(rep_gen) > 0 and rep_gen[0] in element and any(rep_gen[a] in element for a in range(1, len(rep_gen))):
        for e in element: 
            repas.append(dic_alim[int(e.replace("'",""))])
        print(repas)

['couscous royal à la viande', 'eau du robinet', 'café noir prêt à boire non sucré', 'soda au cola light type coca-cola light ou pepsi-cola light', 'sucre blanc ajouté au service']
['couscous royal à la viande', 'eau du robinet', 'café expresso non sucré', 'sucre blanc']
['salade verte sans assaisonnement sans précision', 'vinaigrette sans précision', 'couscous royal à la viande', 'crème dessert rayon frais ou appertisée sans précision', 'yaourt ou spécialité laitière aux fruits sans précision', 'eau du robinet']
['champagne', 'eau du robinet', 'pain baguette', 'couscous royal à la viande', 'salade de fruits frais', 'vin rouge 10°']
['couscous royal à la viande', 'pain baguette', 'eau du robinet']
['contrex eau plate', 'tomate crue', 'couscous royal à la viande', 'café noir prêt à boire non sucré', 'pain baguette', "huile d'olive ajoutée au service"]
['couscous royal à la viande', 'pain baguette', 'banane fraîche', 'évian eau plate']
['saucisson sec', 'couscous royal à la viande', 'emm

In [37]:
# Liste des repas qui contiennent le mot d'entrée mais aucun des autres mots générés
rep_gen=liste_de_chaines
for element in REP_vraisalimfreqliste:
    repas=[]
    if len(rep_gen) > 0 and rep_gen[0] in element and (not any(rep_gen[a] in element for a in range(1, len(rep_gen)))):
        for e in element: 
            repas.append(dic_alim[int(e.replace("'",""))])
        print(repas)

['couscous royal à la viande', 'soda au cola light type coca-cola light ou pepsi-cola light', 'banane fraîche']
['couscous royal à la viande', 'vin rouge 12°', 'fromage sans précision', 'gâteau au chocolat']
['couscous royal à la viande', 'eau de source']
['couscous royal à la viande', 'eau de source']
['couscous royal à la viande', 'pomme non pelée fraîche']
['couscous royal à la viande', 'eau de source']
['couscous royal à la viande', 'eau de source']
['soupe aux légumes préemballée à réchauffer', 'couscous royal à la viande', 'mâche crue', 'raisin blanc frais', 'vinaigrette allégée ajoutée au service']
['couscous royal à la viande', 'soda au cola type coca-cola ou pepsi-cola', 'camembert sans précision']
['couscous royal à la viande', 'soda au cola type coca-cola ou pepsi-cola']
['couscous royal à la viande', 'vittel eau plate', 'raisin blanc frais', 'clémentine ou mandarine fraîche']
['couscous royal à la viande', 'yaourt brassé nature au lait entier type danone velouté nature', 's